# 对比

In [5]:
import time
import sys
import os
import numpy as np
import faiss
import timeit

In [6]:
d = 64                            
nb = 100000                         
nq = 10000                          
np.random.seed(1234)                
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.   
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

## IndexFlatL2

In [7]:
index = faiss.IndexFlatL2(d)        

print('is_trained',index.is_trained)
index.add(xb)                       
print('is_trained',index.is_trained) 

k = 4                               # k=4 k邻近搜索
D, I = index.search(xb[:5], k)          # 执行搜索
print(I[:5])                        # 最初五次查询的结果

is_trained True
is_trained True
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]


IndexFlatL2 是精确暴力搜索，后面的索引跟它相比即可。

## IndexIVFFlat-速度更快

- 将数据集分割成多个，我们在d维空间中定义Voronoi单元，每个数据库向量落在其中一个单元格中。
- 在搜索时，只有查询x所在的单元格中包含的数据库向量y和几个相邻的数据库向量y与查询向量进行比较。

In [8]:
nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
# here we specify METRIC_L2, by default it performs inner-product search

print('is_trained',index.is_trained)
index.train(xb)
print('is_trained',index.is_trained) 

index.add(xb)                  # 添加索引可能会有一点慢
D, I = index.search(xb[:5], k)     # 搜索
print(I[:5])                   # 最初五次查询的结果

is_trained False
is_trained True
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]


结果和上面的强力搜索类似，但是不同。这是因为一些结果不在完全相同的Voronoi。因此，访问更多的单元格可能是有用的。

In [9]:
index.nprobe = 10              # 默认 nprobe 是1 ,可以设置的大一些试试
D, I = index.search(xb[:5], k)
print(I[:5])                  # 最初五次查询的结果

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]


## IndexIVFPQ

In [10]:
nlist = 100
m = 8
k = 4
quantizer = faiss.IndexFlatL2(d)    # 内部的索引方式依然不变
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8) # 每个向量都被编码为8个字节大小

index.train(xb)
index.add(xb)

# D, I = index.search(xb[:5], k)      # 测试
# print(I)
# print(D)

index.nprobe = 10                   # 与以前的方法相比
D, I = index.search(xb[:5], k)          
print(I[:5])                        # 最初五次查询的结果

[[   0   78  608  159]
 [   1 1063  555  380]
 [   2  304  134   46]
 [   3   64  773  265]
 [   4  288  827  531]]
